In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [27]:
data = pd.read_csv(r'D:\Năm 3\Hk2\Bigdata\DoAnCuoiKy\london_bikes.csv')

data.head()

,timestamp,count,temp_real_C,temp_C_feel_like,humidity_percent,wind_speed,weather,is_holiday,is_weekend,season
0,1/4/2015 0:00,182,3.0,2.0,0.930,6.0,Broken clouds,0,1,winter
1,1/4/2015 1:00,138,3.0,2.5,0.930,5.0,Clear,0,1,winter
2,1/4/2015 2:00,134,2.5,2.5,0.965,0.0,Clear,0,1,winter
3,1/4/2015 3:00,72,2.0,2.0,1.000,0.0,Clear,0,1,winter
4,1/4/2015 4:00,47,2.0,0.0,0.930,6.5,Clear,0,1,winter


In [28]:
data.isnull().sum()

timestamp           0
count               0
temp_real_C         0
temp_C_feel_like    0
humidity_percent    0
wind_speed          0
weather             0
is_holiday          0
is_weekend          0
season              0
dtype: int64

In [29]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   timestamp         17414 non-null  object 
 1   count             17414 non-null  int64  
 2   temp_real_C       17414 non-null  float64
 3   temp_C_feel_like  17414 non-null  float64
 4   humidity_percent  17414 non-null  float64
 5   wind_speed        17414 non-null  float64
 6   weather           17414 non-null  object 
 7   is_holiday        17414 non-null  int64  
 8   is_weekend        17414 non-null  int64  
 9   season            17414 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 1.3+ MB


In [30]:
data["timestamp"] = pd.to_datetime(data["timestamp"])


In [31]:
data['timestamp']


0       2015-01-04 00:00:00
1       2015-01-04 01:00:00
2       2015-01-04 02:00:00
3       2015-01-04 03:00:00
4       2015-01-04 04:00:00
                ...        
17409   2017-01-03 19:00:00
17410   2017-01-03 20:00:00
17411   2017-01-03 21:00:00
17412   2017-01-03 22:00:00
17413   2017-01-03 23:00:00
Name: timestamp, Length: 17414, dtype: datetime64[ns]

In [32]:
train = data.iloc[:17000,1:2]
test = data.iloc[17000:17414, 1:2]

In [33]:
print(train.shape)
print(test.shape)

(17000, 1)
(414, 1)


### Chuẩn hoá dữ liệu

In [34]:
from statsmodels.tsa.stattools import pacf
pacf = pacf(data['count'], nlags=20, method='ols')
print(pacf)

[ 1.          0.78148357 -0.41172684  0.18486682 -0.07510114 -0.09947694
 -0.08494917 -0.00403993  0.0920456   0.07065422 -0.32935188 -0.06873344
  0.05112775  0.04852911  0.06695059 -0.19621297 -0.07610221  0.14365297
 -0.11638129  0.18990804  0.16480713]


In [35]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
train_scaled = sc.fit_transform(train)

In [36]:
train_scaled


array([[0.02315522],
       [0.01755725],
       [0.01704835],
       ...,
       [0.17468193],
       [0.16348601],
       [0.14083969]])

In [37]:
train_scaled.shape


(17000, 1)

In [38]:
x_train = []
y_train = []

for i in range(1, 17000):
    x_train.append(train_scaled[i-1:i, 0])
    y_train.append(train_scaled[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [39]:
x_train.shape


(16999, 1)

In [40]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
x_train.shape

(16999, 1, 1)

### Tạo Model

In [41]:
from keras.models import Sequential
from keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense, Activation

In [42]:
rnn = Sequential()
rnn.add(SimpleRNN(activation='relu', units=6, input_shape=(1,1)))
rnn.add(Dense(activation='linear', units=1))


-  Đối tượng Sequential cho phép chúng ta xây dựng một mạng nơ-ron tuần tự bằng cách thêm các lớp lần lượt.
- Lớp SimpleRNN được sử dụng để tạo ra các tầng RNN đơn giản với các đơn vị (units) là 6. Tham số activation='relu' chỉ ra hàm kích hoạt ReLU sẽ được sử dụng. Đối số input_shape=(1,1) chỉ ra kích thước đầu vào là (1,1), tức là có một thời điểm (time step) và một đặc trưng (feature).
- Thêm một lớp Dense vào mô hình. Lớp Dense là một lớp kết nối đầy đủ trong mạng nơ-ron. Tham số units=1 chỉ ra rằng lớp này có một đơn vị đầu ra. Tham số activation='linear' chỉ ra hàm kích hoạt tuyến tính sẽ được sử dụng.

In [43]:
print(rnn.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 6)                 48        
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 55 (220.00 Byte)
Trainable params: 55 (220.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [44]:
rnn.compile(loss='mse', optimizer='adam', metrics=['mse'])
rnn.fit(x_train, y_train, batch_size=1, epochs=2)


Epoch 1/2


16999/16999 [==============================] - 27s 2ms/step - loss: 0.0078 - mse: 0.0078
Epoch 2/2
16999/16999 [==============================] - 26s 2ms/step - loss: 0.0077 - mse: 0.0077


-  Tham số loss='mse' chỉ ra rằng hàm mất mát (loss function) được sử dụng là mean squared error (MSE)
- Tham số optimizer='adam' chỉ ra rằng thuật toán tối ưu hóa Adam sẽ được sử dụng
- Tham số metrics=['mse'] chỉ ra rằng độ đo đánh giá mô hình là mean squared error (MSE).
- Tiến hành huấn luyện mô hình với dữ liệu x_train và y_train. Tham số x_train là dữ liệu đầu vào, y_train là nhãn tương ứng
- Tham số batch_size=1 chỉ ra rằng mỗi lần cập nhật trọng số, một mẫu dữ liệu sẽ được sử dụng
- Tham số epochs=2 chỉ ra rằng quá trình huấn luyện sẽ chạy qua tất cả các mẫu dữ liệu hai lần.

In [45]:
inputs = sc.transform(test)
inputs.shape

(414, 1)

- Áp dụng phương pháp chuẩn hóa Min-Max lên dữ liệu test bằng cách gọi phương thức transform của đối tượng sc (MinMaxScaler)

In [46]:
x_test = []
for i in range(1, 415):
    x_test.append(inputs[i-1:i,0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))
x_test.shape

(414, 1, 1)

In [47]:
rnn = rnn.predict(x_test)
rnn = sc.inverse_transform(rnn)

13/13 [==============================] - 0s 1ms/step


In [48]:
test1 = pd.DataFrame(test)
rnn1= pd.DataFrame(rnn)


In [49]:
test = np.array(test)


In [50]:
import pandas as pd

# Create a DataFrame from the predicted and actual values
df = pd.DataFrame({
    'Ngày': data.iloc[17000:17414, 0].values,
    'Thực tế': test.flatten(),
    'Dự đoán': rnn.flatten()
})

# Export the DataFrame to an Excel file
df.to_excel('du_doan_gia_thuc_te.xlsx', index=False)

plt.figure(figsize=(15,5))
plt.plot(test, marker='.', label='count', color='black')
plt.plot(rnn, marker=',', label='RNN', color='red')
plt.legend()

In [ ]:
mse = np.mean((test1 - rnn1) ** 2)
